In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df= pd.read_csv(f"D:\credit_risk_data\loan.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_10048\2823035199.py:1: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(f"D:\credit_risk_data\loan.csv")


In [3]:
pd.set_option('display.max_columns', None)

## Data Preparation

In [4]:
# Drop unneccesary columns
df.drop(columns = ['application_type', 'addr_state', 'pymnt_plan', 'home_ownership'], inplace = True)
# Drop datetime columns since they dont have too much values in modeling process
df.drop(columns = ['url', 'desc', 'policy_code', 'earliest_cr_line', 'issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d',
                   'id', 'title', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'zip_code'], inplace = True)

In [5]:
# sub_grade — contains redundant information that is already in the grade column
# emp_title — requires other data and a lot of processing to become potentially useful
df.drop(columns = ['sub_grade', 'emp_title'], inplace = True)

In [6]:
df = df[((df.loan_status == 'Current') | ( df.loan_status == 'Fully Paid') | (df.loan_status == 'Charged Off') | (df.loan_status == 'Default')
         | (df.loan_status == 'Does not meet the credit policy. Status:Fully Paid') | (df.loan_status == 'Does not meet the credit policy. Status:Charged Off'))]

In [7]:
# convert target variable to binary
for i in df['loan_status'].unique():
    if (i == 'Current'):
        df['loan_status'].replace({i:0}, inplace = True)
    elif (i == 'Does not meet the credit policy. Status:Fully Paid'):
        df['loan_status'].replace({i:0}, inplace = True)
    elif (i == 'Fully Paid'):
        df['loan_status'].replace({i:0}, inplace = True)
    else:
        df['loan_status'].replace({i:1}, inplace = True)


In [8]:
df.loan_status.value_counts()

0    811490
1     47228
Name: loan_status, dtype: int64

### NA treatment

#### Drop NaN

In [9]:
#drop columns having na rate > 75%
na_rates = df.isna().sum()/len(df)
drop_columns = na_rates[na_rates > 0.75].index
df.drop(columns = drop_columns, inplace = True)

In [10]:
df.initial_list_status.value_counts()

f    445304
w    413414
Name: initial_list_status, dtype: int64

#### Fill NaN

In [11]:
dict = pd.read_csv("D:\credit_risk_data\LCDataDictionary.csv")

In [12]:
na_rate_col = pd.DataFrame(df.isna().sum()/len(df), columns = ['na_rate'])
na_rate_col = na_rate_col[na_rate_col['na_rate'] > 0]

In [13]:
pd.options.display.max_colwidth = 150

In [14]:
a = pd.merge(na_rate_col, dict, 'inner', left_index = True, right_on = 'LoanStatNew')
data_type = pd.DataFrame(df[na_rate_col].dtypes, columns = ['type'])
na_col_info = pd.merge(a, data_type, left_on = a['LoanStatNew'], right_index = True, how = 'left')
na_col_info = na_col_info.sort_values('na_rate', ascending = False).set_index('LoanStatNew')

In [15]:
na_col_info

,na_rate,Description,Unnamed: 2,type
LoanStatNew,,,,
mths_since_last_delinq,0.513569,The number of months since the borrower's last delinquency.,NaN,float64
tot_coll_amt,0.081589,Total collection amounts ever owed,NaN,float64
tot_cur_bal,0.081589,Total current balance of all accounts,NaN,float64
emp_length,0.050185,Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.,NaN,object
revol_util,0.000566,"Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.",NaN,float64
collections_12_mths_ex_med,0.000169,Number of collections in 12 months excluding medical collections,NaN,float64
delinq_2yrs,0.000034,The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years,NaN,float64
inq_last_6mths,0.000034,The number of inquiries in past 6 months (excluding auto and mortgage inquiries),NaN,float64
open_acc,0.000034,The number of open credit lines in the borrower's credit file.,NaN,float64


In [16]:
# Na in emp_length columns means that customer have no job ==> fill na by "< 1 year"
df['emp_length'].fillna('< 1 year', inplace = True)

In [17]:
for i in ['mths_since_last_delinq', 'open_acc', 'pub_rec', 'annual_inc', 'inq_last_6mths', 'total_acc', 'total_rev_hi_lim']:
    df[i].fillna(df[i].median(), inplace = True)
for a in ['tot_coll_amt', 'tot_cur_bal', 'revol_util', 'collections_12_mths_ex_med', 'delinq_2yrs', 'acc_now_delinq']:
    df[a].fillna(0, inplace = True)

In [18]:
df_cleaned = df.copy()

In [19]:
df_cleaned.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
count,8.587180e+05,858718.000000,858718.000000,858718.00000,858718.000000,858718.000000,8.587180e+05,858718.000000,858718.000000,858718.000000,858718.000000,858718.000000,858718.000000,858718.000000,8.587180e+05,858718.000000,858718.000000,858718.000000,858718.000000,858718.000000,858718.000000,858718.000000,858718.000000,858718.000000,858718.000000,858718.000000,8.587180e+05,8.587180e+05,8.587180e+05
mean,3.465289e+07,14727.305588,14713.504957,14672.99065,13.195385,435.776571,7.505947e+04,0.054998,18.109521,0.311380,0.692000,32.519318,11.537778,0.194238,1.694413e+04,54.981632,25.260845,8258.308137,8254.938077,7657.445198,7618.613918,47.451867,5.043660,2224.174437,0.014184,0.004933,2.070217e+02,1.283628e+05,3.148120e+04
std,2.403397e+07,8425.407323,8419.630810,8432.27645,4.364431,243.753081,6.427936e+04,0.227977,17.402105,0.856858,0.997503,15.341504,5.309839,0.580992,2.245853e+04,23.857277,11.827448,8461.344017,8457.894007,7907.847058,7882.600659,416.387518,64.163684,4860.838033,0.133496,0.077240,1.005262e+04,1.524055e+05,3.620144e+04
min,7.047300e+04,500.000000,500.000000,0.00000,5.320000,15.670000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.075776e+07,8000.000000,8000.000000,8000.00000,9.990000,260.100000,4.500000e+04,0.000000,11.870000,0.000000,0.000000,31.000000,8.000000,0.000000,6.456000e+03,37.600000,17.000000,0.000000,0.000000,1972.135000,1957.930000,0.000000,0.000000,285.080000,0.000000,0.000000,0.000000e+00,2.296000e+04,1.480000e+04
50%,3.667514e+07,13000.000000,13000.000000,12950.00000,12.990000,382.325000,6.500000e+04,0.000000,17.600000,0.000000,0.000000,31.000000,11.000000,0.000000,1.189500e+04,55.900000,24.000000,6237.925000,6235.440000,4983.500000,4945.580000,0.000000,0.000000,468.590000,0.000000,0.000000,0.000000e+00,6.517450e+04,2.380000e+04
75%,5.800390e+07,20000.000000,20000.000000,20000.00000,15.990000,571.140000,9.000000e+04,0.000000,23.880000,0.000000,1.000000,31.000000,14.000000,0.000000,2.085900e+04,73.500000,32.000000,13502.650000,13493.870000,10750.890000,10691.830000,0.000000,0.000000,850.200000,0.000000,0.000000,0.000000e+00,1.958990e+05,3.790000e+04
max,7.351969e+07,35000.000000,35000.000000,35000.00000,28.990000,1445.460000,9.500000e+06,1.000000,9999.000000,39.000000,33.000000,188.000000,90.000000,86.000000,2.904836e+06,892.300000,169.000000,49372.860000,49372.860000,57777.579870,57777.580000,33520.270000,7002.190000,36475.590000,20.000000,14.000000,9.152545e+06,8.000078e+06,9.999999e+06


## Single Factor Analysis (SFA)

In [20]:
def WOE(colname):
    a = df_cleaned[['member_id', 'loan_status', colname]]
    coltype = df_cleaned[colname].dtype
    if coltype in ['float', 'int']:
        a = pd.DataFrame(pd.qcut(df_cleaned[colname].sort_values(), 6, duplicates='drop'))
    a['loan_status'] = df_cleaned['loan_status']
    bin_table = a.groupby(colname).agg({'loan_status':['count','sum']})
    bin_table.index.names = ['Cut_point']
    bin_table.columns = ['Total', '#BAD']
    bin_table['#GOOD'] = bin_table['Total'] - bin_table['#BAD']
    bin_table['Per_bin(%)'] = (bin_table['Total']/bin_table['Total'].sum())* 100
    bin_table['Bad_rate'] = bin_table['#BAD']/bin_table['#BAD'].sum()
    bin_table['Good_rate'] = bin_table['#GOOD']/bin_table['#GOOD'].sum()
    bin_table['WOE'] = np.log(bin_table['Good_rate']/bin_table['Bad_rate'])
    return bin_table


def IV(colname):
    bin_table = WOE(colname)
    bin_table['IV'] = (1-2*bin_table['Bad_rate'])*bin_table['WOE']
    return bin_table['IV'].sum()

In [21]:
variables = pd.Series(list(set(df_cleaned.columns).difference({'loan_status', 'member_id'})))
IV = pd.DataFrame(variables.map(lambda x: IV(x)), columns = ['IV']).set_index(variables)

C:\Users\Admin\AppData\Local\Temp\ipykernel_10048\3410162551.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['loan_status'] = df_cleaned['loan_status']
C:\Users\Admin\AppData\Local\Temp\ipykernel_10048\3410162551.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['loan_status'] = df_cleaned['loan_status']
C:\Users\Admin\AppData\Local\Temp\ipykernel_10048\3410162551.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [22]:
IV = IV.sort_values('IV', ascending = False)
#Define shortlisted variables by choosing variables having IV >= 0.05
final_IV = IV[IV['IV'] >= 0.05]

In [23]:
final_IV

,IV
out_prncp_inv,14.364685
out_prncp,14.364080
last_pymnt_amnt,3.615113
int_rate,1.782529
total_rev_hi_lim,0.699687
tot_cur_bal,0.574552
total_pymnt,0.459771
initial_list_status,0.435466
total_pymnt_inv,0.423399
annual_inc,0.323330


In [24]:
columns_in_df_WOE = list(final_IV.index)
columns_in_df_WOE.append('member_id')
columns_in_df_WOE.append('loan_status')

df_WOE = df_cleaned[columns_in_df_WOE]
for feature in final_IV.index:

    # Calculate WOE for quantile_bins
    bin_table = WOE(feature)
    
    # Create a dictionary to map bin to WOE
    woe_map = bin_table['WOE'].to_dict()
    
    # Map the WOE values to the original data
    df_WOE[f'{feature}_woe'] = df_WOE[f'{feature}'].map(woe_map)
    df_WOE.drop(columns = [f'{feature}'], inplace = True)
df_WOE.set_index('member_id', inplace = True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_10048\2130185705.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_WOE[f'{feature}_woe'] = df_WOE[f'{feature}'].map(woe_map)
C:\Users\Admin\AppData\Local\Temp\ipykernel_10048\2130185705.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_WOE.drop(columns = [f'{feature}'], inplace = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_10048\2130185705.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

## Correlation Analysis

In [25]:
# define target variable
target = 'loan_status'
model_features = list(set(df_WOE.columns).difference({target}))

X = df_WOE[model_features]
y = df_WOE[target]

In [26]:
cor_matrix = X.corr()
threshold = 0.5
corr_pairs = cor_matrix.unstack()
high_corr_pairs = corr_pairs[(corr_pairs > threshold) & (corr_pairs < 1)]
high_corr_pairs

total_pymnt_woe      total_pymnt_inv_woe    0.988089
out_prncp_woe        out_prncp_inv_woe      0.999844
out_prncp_inv_woe    out_prncp_woe          0.999844
open_acc_woe         total_acc_woe          0.595936
total_pymnt_inv_woe  total_pymnt_woe        0.988089
total_acc_woe        open_acc_woe           0.595936
dtype: float64

## Modeling

In [27]:
# Take 1 variable in each group, start with highest IV variable
X1 = X.drop(columns = ['out_prncp_woe', 'total_pymnt_inv_woe', 'total_acc_woe'])
X2 = X.drop(columns = ['out_prncp_woe', 'total_pymnt_inv_woe', 'open_acc_woe'])
X3 = X.drop(columns = ['out_prncp_woe', 'total_pymnt_woe', 'total_acc_woe'])
X4 = X.drop(columns = ['out_prncp_inv_woe', 'total_pymnt_woe', 'total_acc_woe'])
X5 = X.drop(columns = ['out_prncp_inv_woe', 'total_pymnt_inv_woe', 'total_acc_woe'])
X6 = X.drop(columns = ['out_prncp_inv_woe', 'total_pymnt_inv_woe', 'open_acc_woe'])
X7 = X.drop(columns = ['out_prncp_woe', 'total_pymnt_woe', 'total_acc_woe'])
X8 = X.drop(columns = ['out_prncp_inv_woe', 'total_pymnt_woe', 'open_acc_woe'])

### Logistic regression

In [28]:
import statsmodels
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [29]:
threshold = 0.7
gini = []
for X in [X1, X2, X3, X4, X5, X6, X7, X8]:
    # Data Standardization
    scaler = StandardScaler()
    scaler.fit(X)

    X = scaler.transform(X)
    y = df_WOE['loan_status']
    #Train, test splitting
    x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size=0.3)

    #Regression
    lr = LogisticRegression(solver='newton-cg', class_weight='balanced', random_state = 0)     
    lr.fit(x_train, y_train)            
        
    # Predicting on the train data
    pred_test = lr.predict_proba(x_train)
    pred_test = pd.DataFrame(pred_test,
                             columns = ['0', '1'])      # Create table of estimated results
        
    # convert to binary results based on threshold
    pred_test_binary = pd.DataFrame(np.where(pred_test['1'] >= threshold, 1, 0), columns = ['1']) 
    #Calculating and printing the f1 score 
    
    gini_score = 2*roc_auc_score(y_train, pred_test['1']) - 1
    gini.append(gini_score)

C:\Users\Admin\anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
C:\Users\Admin\anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:415: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
C:\Users\Admin\anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


In [30]:
#Finding submodel havng highest gini score
summary = pd.DataFrame(gini, columns = ['gini'])
summary['sub_models'] = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8']
summary[summary['gini'] == summary['gini'].max()]

,gini,sub_models
1,0.91357,X2


## Model Performance

### 1. P-values

In [31]:
import statsmodels
import statsmodels.api as sm 
  
# building the model and fitting the data 
X_added_constant = sm.add_constant(X)
log_reg = sm.Logit(y, X_added_constant).fit()
log_reg.summary()

Optimization terminated successfully.
         Current function value: 0.104061
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            loan_status   No. Observations:               858718
Model:                          Logit   Df Residuals:                   858704
Method:                           MLE   Df Model:                           13
Date:                Tue, 23 Jul 2024   Pseudo R-squ.:                  0.5114
Time:                        21:51:51   Log-Likelihood:                -89359.
converged:                       True   LL-Null:                   -1.8289e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.5650      0.022   -255.717      0.000      -5.608      -5.522
x1            -0.8056      0.008   -105.045      0.000      -0.821      -0.791
x2            -2.6694      0.014   -184.977      0.000      -2.698      -2.641
x3             0.1168      0.007     16.789      0.000       0.103       0.130
x4             0.1397      0.007     20.211      0.000       0.126       0.153
x5             0.0022      0.006      0.350      0.726      -0.010       0.014
x6            -0.1229      0.006    -19.933      0.000      -0.135      -0.111
x7             0.0650      0.007      9.046      0.000       0.051       0.079
x8             0.0571      0.008      7.279      0.000       0.042       0.073
x9            -0.0876      0.006    -13.769      0.000      -0.100      -0.075
x10           -0.0176      0.007     -2.644      0.008      -0.031      -0.005
x11           -1.9173      0.018   -104.291      0.000      -1.953      -1.881
x12           -0.4237      0.007    -64.426      0.000      -0.437      -0.411
x13            0.1610      0.007     24.583      0.000       0.148       0.174
==============================================================================
"""

revol_util_woe is not significant

### 2. Kolmogorov-Smirnov test

In [32]:
#Perfrom KS-test for submodel 2
x_train, x_test, y_train, y_test = train_test_split(X2, y, random_state = 0, test_size=0.3)
lr.fit(x_train, y_train)
y_pred = lr.predict_proba(x_test)
y_pred = y_pred[:, 1]

In [33]:
def FPR_TPR_calculation(y_pred, n_bins):
  _, thresholds = pd.qcut(y_pred, q=n_bins, retbins=True)
  FPR_list = []
  TPR_list = []
  GOOD = set(np.where(y_test == 0)[0])
  BAD = set(np.where(y_test == 1)[0])
  total_BAD = len(BAD)
  total_GOOD = len(GOOD)
  for thres in thresholds:
    pred_id = set(np.where(y_pred >= thres)[0])
    # Đếm % số lượng hồ sơ BAD có xác suất dự báo nhỏ hơn hoặc bằng thres
    TPR = len(pred_id.intersection(BAD))/total_BAD
    TPR_list.append(TPR)
    # Đếm % số lượng hồ sơ GOOD có xác suất dự báo nhỏ hơn hoặc bằng thres
    FPR = len(pred_id.intersection(GOOD))/total_GOOD
    FPR_list.append(FPR)
  FPR_list = np.array(FPR_list)
  TPR_list = np.array(TPR_list)
  return TPR_list, FPR_list, thresholds, pred_id

TPR_list, FPR_list, thresholds, pred_id = FPR_TPR_calculation(y_pred, n_bins=6)

In [34]:
from scipy import stats

stats.ks_2samp(TPR_list, FPR_list)


KstestResult(statistic=0.7142857142857143, pvalue=0.05303030303030303)

P-values of approximately 0.053 means this model is significant in classifying customers.